In [1]:
import pandas as pd 
import cv2 
import numpy as np 


In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,0,1
0,data/HockeyFights/no/no340_xvid.avi,0
1,data/HockeyFights/yes/fi166_xvid.avi,1
2,data/HockeyFights/no/no220_xvid.avi,0
3,data/HockeyFights/no/no365_xvid.avi,0
4,data/HockeyFights/yes/fi249_xvid.avi,1


In [4]:
x = df.iloc[:,0].values
y = df.iloc[:,1].values

In [5]:
                previous= previous_frame[int(bbox[1]):int(bbox[3]),int(bbox[0]):int(bbox[2])]
#                 cv2.imshow('dd1', previous)
#                 cv2.waitKey(1)
                current = frame[int(bbox[1]):int(bbox[3]),int(bbox[0]):int(bbox[2])]
#                 cv2.imshow('dd2', current)
#                 cv2.waitKey(1)
                prvs = cv2.cvtColor(previous,cv2.COLOR_BGR2GRAY)
                hsv = np.zeros_like(previous)
                hsv[...,1] = 255
                next = cv2.cvtColor(current,cv2.COLOR_BGR2GRAY)
                next = next.reshape(prvs.shape)
                flow = cv2.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                mags.append(mag[int((mag.shape[0]-1)/2),int((mag.shape[1]-1)/2)])
            
                #print(mag.shape[0])
                #print(mag[mag.shape(0),mag.shape(1)])

                hsv[...,0] = ang*180/np.pi/2
                dac = ang*180/np.pi/2
                angs.append(dac[int((dac.shape[0]-1)/2),int((dac.shape[1]-1)/2)])
                hsv[...,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                tac = "mag:" +str(mag[int((mag.shape[0]-1)/2),int((mag.shape[1]-1)/2)])+" angle:"+str(dac[int((dac.shape[0]-1)/2),int((dac.shape[1]-1)/2)])

                cv2.imshow('frame2',rgb)

NameError: name 'previous_frame' is not defined

In [12]:
for i in x:
    cap = cv2.VideoCapture(i)
    frame_count=0
    previous_frame = 0
    o_mag=0
    o_ang=0
    
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret == True:
            if frame_count>0:
                gray1 = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                gray2 = cv2.cvtColor(previous_frame, cv2.COLOR_BGR2GRAY)
                gray1 = gray1.reshape(gray2.shape)
                flow = cv2.calcOpticalFlowFarneback(gray2,gray1, None, 0.5, 3, 15, 3, 5, 1.2, 0)
                mag, ang = cv2.cartToPolar(flow[...,0], flow[...,1])
                magnitude = mag - o_mag
                angle = ang-o_ang
                o_ang = ang 
                o_mag = mag
                
                
                print(magnitude.shape)
                print(angle.shape)
                hsv = np.zeros_like(previous_frame)
                hsv[...,1] = 255
                hsv[...,0] = angle*180/np.pi/2
                hsv[...,2] = cv2.normalize(magnitude,None,0,255,cv2.NORM_MINMAX)
                rgb = cv2.cvtColor(hsv,cv2.COLOR_HSV2BGR)
                cv2.imshow('frame2',rgb)
            
#             cv2.imshow('frame',gray)
            previous_frame = frame 
            frame_count = frame_count+1
            
            if cv2.waitKey(30) & 0xFF == ord('q'):
                break

        else:
            break

    cap.release()
    #out.release()
    cv2.destroyAllWindows() 

(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)


D:\anaconda\envs\testing_env\lib\site-packages\ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in subtract


(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)

(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)
(288, 360)

KeyboardInterrupt: 